In [2]:
from TexSoup import TexSoup
import shutil
import re
import os
import difflib
import pprint

#os.path.exists(os.path.join(os.getcwd(), 'src/pages/sta'))

In [9]:
coursetags = {
    'Statics': 'sta',
    'Dynamics': 'dyn',
    'Solid_Mechanics': 'sol'
}

course = 'Dynamics'

tex_files = list(sorted([file[:-4] for file in os.listdir(f'./public/{course}') if file[-4:] == '.tex' and (file != 'main.tex')]))

tex_files

if not os.path.exists(f'./src/pages/{coursetags[course]}'):
    os.mkdir(f'./src/pages/{coursetags[course]}')

In [10]:
replacements = {
    'section': 'Section',
    'subsection': 'SubSection',
    'subsubsection': 'SubSubSection',
    'blue': 'BlueText',
    'red': 'RedText',
    'textit': 'em',
    'textbf': 'strong'
}

In [15]:
for filename in tex_files:
    required_modules = ['Image']

    tex_path = f"public/{course}/{filename}.tex"

    astro_path = f"src/pages/{coursetags[course]}/{filename.lower().replace(' ', '_')}.astro"


    with open(tex_path, 'r+') as file:
        raw = file.read()

        data = TexSoup(raw)
        for tex, html in replacements.items():
            
            matches = data.find_all(tex)
            
            for match in list(matches):
                if tex == 'blue':
                    print(str(match)[:6], filename)

                if len(list(match.contents)) != 0:
                    #print(str(match))
                    
                    #raw = raw.replace(r"\%s{%s}" % (tex, list(match.contents)[0]), f"<{html}>{list(match.contents)[0]}</{html}>")

                    raw = raw.replace(str(match).replace(r'\blue {', r'\blue{'), f"<{html}>{' '.join(list([str(e) for e in match.contents]))}</{html}>")

                    if html != 'em' and html != 'strong' and (html not in required_modules):
                        required_modules.append(html)
        
        # ADDITIONAL CLEANUP

        raw = raw.replace('<Section>', '<Section title="">')
        raw = raw.replace('<SubSection>', '<SubSection title="">')
        raw = raw.replace('<SubSubSection>', '<SubSubSection title="">')
        #print([e for e in data.subsection.descendants])
        

        raw = raw.replace(r'\noindent', '')
        enum = list(data.find_all('enumerate'))

        if len(enum): 
            required_modules.append('Enumerate')
            required_modules.append('Item')

        for l in enum:
            s = str(l)
            final_str = ''
            for c in l.children:
                final_str += f"\t<Item>{list(c.text)[0]}</Item>\n"
            
            final_str = '<Enumerate>\n'+final_str+'</Enumerate>\n'

            raw = raw.replace(s, final_str)

        item = list(data.find_all('itemize'))

        if len(item): 
            required_modules.append('Itemize')
            if 'Item' not in required_modules:
                required_modules.append('Item')


        for l in item:
            #print(l)
            s = str(l)
            final_str = ''
            for c in l.children:
                final_str += f"\t<Item>{list(c.text)[0]}</Item>\n"
            
            final_str = '<Itemize>\n'+final_str+'</Itemize>\n'

            raw = raw.replace(s, final_str)

        comments = re.findall('\n%(.*)\n', raw)
        
        for comment in comments:
            raw = raw.replace(f'%{comment}', f'<!-- {comment} -->')


        inline_equations = re.findall('\$([^$]*)\$', raw)
        
        if len(inline_equations): required_modules.append('InlineEquation')

        for eq in inline_equations:
            db_backslash = eq.replace('\\','\\\\')
            raw = raw.replace(f'${eq}$', f'<InlineEquation equation="{db_backslash}" />')

        display_equations = re.findall('\[(.*)\]', raw)

        for eq in display_equations:
            db_backslash = eq.replace('\\','\\\\')
            if r'\[%s]'% (eq) in raw and ('DisplayEquation' not in required_modules): required_modules.append('DisplayEquation')
            raw = raw.replace(r'\[%s]'% (eq), f'<DisplayEquation equation="{db_backslash}" />')

        imports = 'import Layout from "../../layouts/Layout.astro" \n'

        for comp in required_modules:
            imports += f'import {comp} from "../../components/{comp}.astro" \n' 

        figs = data.find_all('figure')

        for fig in figs:
            caption = ' '.join([str(e) for e in list(fig.caption.contents)[1:]])
            
            actual_text = caption[6:-1] if caption[0:5] == r'\blue' else caption
            
            inline_equations = re.findall('\$([^$]*)\$', actual_text)

            for eq in inline_equations:
                db_backslash = eq.replace('\\','\\\\')
                actual_text = actual_text.replace(f'${eq}$', f'<InlineEquation equation="{db_backslash}" />')

            if caption[0:5] == r'\blue':
                actual_text = f'<BlueText>{actual_text}</BlueText>'

            src = list(fig.includegraphics.text)[-1]
            try:
                width = list(fig.includegraphics.text)[0].split(',')[1].split('=')[1].replace('in', '')
            except:
                width = '5' 

            #print(actual_text)

            raw = raw.replace(r'\end{figure}', f'<Image src="/{course}/{src}" width="{width}">{actual_text}</Image>', 1)

        figs = data.find_all('figure*')

        for fig in figs:
            caption = ' '.join([str(e) for e in list(fig.caption.contents)[1:]])
            
            actual_text = caption[6:-1] if caption[0:5] == r'\blue' else caption
            
            inline_equations = re.findall('\$([^$]*)\$', actual_text)

            for eq in inline_equations:
                db_backslash = eq.replace('\\','\\\\')
                actual_text = actual_text.replace(f'${eq}$', f'<InlineEquation equation="{db_backslash}" />')

            if caption[0:5] == r'\blue':
                actual_text = f'<BlueText>{actual_text}</BlueText>'

            src = list(fig.includegraphics.text)[-1]
            try:
                width = list(fig.includegraphics.text)[0].split(',')[1].split('=')[1].replace('in', '')
            except:
                width = '5' 

            #print(actual_text)

            raw = raw.replace(r'\end{figure*}', f'<Image src="/{course}/{src}" width="{width}">{actual_text}</Image>', 1)

        #print(imports)

        raw = '---\n' + imports + '---\n' + f'<Layout title="{filename}">' + '\n' + raw + '\n' +'</Layout>'
        

        astro_file = open(astro_path, mode='w+')

        astro_file.write(raw)

        astro_file.close()



\blue{ Contact and Rolling
\blue{ Contact and Rolling
\blue{ Contact and Rolling
\blue{ Particle Kinematics
\blue{ Particle Kinematics
\blue{ Particle Kinematics
\blue{ Particle Kinematics
\blue{ Particle Kinematics
\blue{ Particle Kinematics
\blue{ Particle Kinematics
\blue{ Particle Kinematics
\blue{ Particle Kinematics
\blue{ Particle Kinetics
\blue{ Particle Kinetics
\blue{ Particle Kinetics
\blue{ Particle Kinetics
\blue{ Particle Kinetics
\blue{ Particle Kinetics
\blue{ Particle Kinetics
\blue{ Rigid Body Kinematics
\blue{ Rigid Body Kinematics
\blue{ Rigid Body Kinematics
\blue{ Rigid Body Kinematics
\blue{ Rigid Body Kinematics
\blue{ Rigid Body Kinematics
\blue{ Rigid Body Kinematics
\blue{ Rigid Body Kinematics
\blue{ Rigid Body Kinetics
\blue{ Rigid Body Kinetics
\blue{ Rigid Body Kinetics
\blue{ Rigid Body Kinetics
\blue{ Rigid Body Kinetics
\blue{ Rigid Body Kinetics
\blue{ Vector Calculus
\blue{ Vector Calculus
\blue{ Vector Calculus
\blue{ Vector Calculus
\blue{ Vector C

In [34]:
re.findall('\$([^$]*)\$', r'ration $\vec{a}$ of a point mass with mass $m$ to the total applied force $\vec{F}$ on the mass ')

['\\vec{a}', 'm', '\\vec{F}']